In [4]:
from google.colab import drive
import os
import cv2
import numpy as np
import tensorflow as tf
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import zipfile
zip_file_path = '/content/drive/MyDrive/Pred Img.zip'
extract_path = '/content/Img'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [2]:
! pip install mtcnn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 29.8 MB/s eta 0:00:00


In [9]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from mtcnn import MTCNN

# Function to preprocess face image for prediction
def preprocess_face(face_img, target_size=(224, 224)):
    face_img = cv2.resize(face_img, target_size)
    face_img = face_img / 255.0  # Normalize to [0, 1]
    face_img = np.expand_dims(face_img, axis=0)  # Add batch dimension
    return face_img

# Function to expand the bounding box
def expand_bbox(bbox, expansion_factor=0.3):
    x, y, w, h = bbox
    new_w = int(w * (1 + expansion_factor))
    new_h = int(h * (1 + expansion_factor))
    new_x = max(0, x - (new_w - w) // 2)
    new_y = max(0, y - (new_h - h) // 2)
    return new_x, new_y, new_w, new_h

# Load the trained model
model_path = '/content/drive/My Drive/face_mask_detection_model_1pt4.h5'  # Update with the actual path to your model
model = tf.keras.models.load_model(model_path)

# Class names
class_names = ['with_mask', 'without_mask']

# Initialize the face detector with adjusted parameters
detector = MTCNN(
    min_face_size=15,  # Adjust as needed
    scale_factor=0.5,  # Adjust as needed
    steps_threshold=[0.4, 0.4, 0.3]  # Lower values for more sensitivity
)

# Define the folder path
folder_path = '/content/Img/Pred Img/'

# List to store images and labels
images_with_predictions = []

# Loop through all images in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".jpeg") or filename.endswith(".png"):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)

        if img is not None:
            # Detect faces in the image
            faces = detector.detect_faces(img)
            for face in faces:
                x, y, w, h = face['box']
                x, y, w, h = expand_bbox((x, y, w, h), expansion_factor=0.3)  # Adjust the expansion factor as needed
                face_img = img[y:y+h, x:x+w]

                if face_img.size == 0:
                    continue

                # Preprocess the face image
                face_img = preprocess_face(face_img)

                # Predict using the trained model
                pred = model.predict(face_img)
                confidence = pred[0][0]
                class_idx = int(confidence > 0.5)
                class_name = class_names[class_idx]

                # Draw bounding box and label on the image
                color = (0, 255, 0) if class_idx == 0 else (0, 0, 255)
                label = f"{class_name}: {confidence:.2f}"
                cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
                cv2.putText(img, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

            # Convert BGR image to RGB for displaying with matplotlib
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            images_with_predictions.append((img_rgb, label))

# Display the images in a grid
def plot_images(images, labels, cols=4, rows=3):
    fig, axes = plt.subplots(rows, cols, figsize=(20, 15))
    axes = axes.flatten()
    for img, ax, label in zip(images, axes, labels):
        ax.imshow(img)
        ax.set_title(label)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

print(images_with_predictions)

# Extract images and labels from the list
images, labels = zip(*images_with_predictions)

# Plot the images with predictions
plot_images(images, labels, cols=4, rows=3)


Output hidden; open in https://colab.research.google.com to view.

In [5]:
import cv2
import numpy as np
from mtcnn import MTCNN
from google.colab.patches import cv2_imshow
import tensorflow as tf

# Function to expand the bounding box
def expand_bbox(bbox, expansion_factor=0.3):
    x, y, w, h = bbox
    new_w = int(w * (1 + expansion_factor))
    new_h = int(h * (1 + expansion_factor))
    new_x = max(0, x - (new_w - w) // 2)
    new_y = max(0, y - (new_h - h) // 2)
    return new_x, new_y, new_w, new_h

# Function to preprocess face image for prediction
def preprocess_face(face_img, target_size=(224, 224)):
    face_img = cv2.resize(face_img, target_size)
    face_img = face_img / 255.0  # Normalize to [0, 1]
    face_img = np.expand_dims(face_img, axis=0)  # Add batch dimension
    return face_img

# Initialize the face detector with adjusted parameters
detector = MTCNN(
    min_face_size=15,  # Adjust as needed
    scale_factor=0.5,  # Adjust as needed
    steps_threshold=[0.4, 0.4, 0.3]  # Lower values for more sensitivity
)

# Load the trained model
model = tf.keras.models.load_model('/content/drive/My Drive/face_mask_detection_model_1pt4.h5')
class_names = ['with_mask', 'without_mask']

# Define a confidence threshold for uncertain predictions
confidence_threshold = 0.40

# Path to the video file or 0 for webcam
video_path = '/content/drive/My Drive/VID3.webm'

# Open the video file
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Detect faces in the frame
    faces = detector.detect_faces(frame)
    for face in faces:
        x, y, w, h = face['box']
        x, y, w, h = expand_bbox((x, y, w, h), expansion_factor=0.3)  # Adjust the expansion factor as needed
        face_img = frame[y:y+h, x:x+w]

        if face_img.size == 0:
            continue

        # Preprocess the face image
        face_img = preprocess_face(face_img)

        # Predict using the trained model
        pred = model.predict(face_img)
        confidence = pred[0][0]
        class_idx = int(confidence > 0.5)

        # Debug: print prediction
        print(f"Prediction: {pred[0][0]}, Class Index: {class_idx}")

        # Check if prediction is uncertain
        if confidence > confidence_threshold and confidence < (1 - confidence_threshold) or confidence == None:
            label = f"Uncertain: {confidence:.2f}"
            color = (0, 255, 255)  # Yellow color for uncertain predictions
        else:
            # Get the class name
            class_name = class_names[class_idx]
            color = (0, 255, 0) if class_idx == 0 else (0, 0, 255)
            label = f"{class_name}: {confidence:.2f}"

        # Draw bounding box and label on the frame
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # Display the frame
    cv2_imshow(frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video file and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

Output hidden; open in https://colab.research.google.com to view.